In [21]:
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster, DirectTimeSeriesRegressionForecaster, DirectTabularRegressionForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.compose import YfromX
from sktime.transformations.series.boxcox import LogTransformer
from sktime.transformations.compose import ColumnwiseTransformer
from sktime.forecasting.compose import ForecastingPipeline

import warnings
warnings.filterwarnings('ignore')

## Direct Strategy V3
Adding more lags to the model


In [22]:
price_old_df = pd.read_csv('https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
price_old_df = price_old_df.asfreq('H')
price_old_df = price_old_df.sort_values(by='Date (MST)')
price_old_df = price_old_df['2022-12':]
price_df = price_old_df['price']

Math:

To predict for a forecast horizon of 4 steps in the future, we need to create 4 models.
For each model, let's say that we include 3 lags of price as the training data.

So, in the below code, we need to create a sliding window with window length as (4+3-1) = 6. 

- It will be like:
- 1 2 3 4 5 6 T

- Model one will have features as 1 2 3 and target as T: Four step ahead prediction
- Model two will have features as 2 3 4 and target as T: Three step ahead prediction
- Model three will have features as 3 4 5 and target as T: Two step ahead prediction
- Model four will have features as 4 5 6 and target as T: Next step prediction 


For this example, I am interested in predicting the next 3 steps. So, I will create 3 models.
Also, for each model I want to include 2 lags each for features. 
So, I should pass a sliding window of length (3+2-1) = 4.

- It will be like:
- 1 2 3 4 T

- Model one will have features as 1 2 and target as T: Two step ahead prediction
- Model two will have features as 2 3 and target as T: One step ahead prediction
- Model three will have features as 3 4 and target as T: Next step prediction

_______________

For this example, I am interested in predicting the next 12 steps. So, I will create 12 models.
Also, for each model I want to include 24 lags of price. 
So, I should pass a sliding window of length (12+24-1) = 35.



In [23]:
def create_sliding_window_data_refined(ts, window_length):
    # Create a SlidingWindowSplitter object
    splitter = SlidingWindowSplitter(fh = [1], window_length=window_length, step_length=1)
    # Split the series using the splitter
    split_series = list(splitter.split_series(ts))
    #Create an empty DataFrame to store the data
    data = pd.DataFrame()
    # Iterate over the split series and extract the features and target
    for i, (train, test) in enumerate(split_series):
        # Extract features and target from the split series
        features = ts[train.index]
        target = ts[test.index]
        split_data = {'date': pd.to_datetime(target.index[0].to_pydatetime())}
    
        for i, j in zip(range(window_length, 0, -1), range(0, window_length)):
            split_data[f'price_lag_{i}'] = features[j]
    
        
        split_data[f'price_target'] = target[0]

        split_df = pd.DataFrame(split_data, index=['date'])

        data = pd.concat([data, split_df])   
    data.set_index('date', inplace=True)
    return data

In [24]:
len_models = 12
num_lags = 24
window_length = num_lags + len_models - 1
price_data_df = create_sliding_window_data_refined(price_df, window_length)
price_data_df.head()

,price_lag_35,price_lag_34,price_lag_33,price_lag_32,price_lag_31,price_lag_30,price_lag_29,price_lag_28,price_lag_27,price_lag_26,...,price_lag_9,price_lag_8,price_lag_7,price_lag_6,price_lag_5,price_lag_4,price_lag_3,price_lag_2,price_lag_1,price_target
date,,,,,,,,,,,,,,,,,,,,,
2022-12-02 11:00:00,87.67,108.46,104.96,212.77,112.69,374.99,509.57,112.67,144.00,622.06,...,106.89,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82
2022-12-02 12:00:00,108.46,104.96,212.77,112.69,374.99,509.57,112.67,144.00,622.06,682.36,...,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13
2022-12-02 13:00:00,104.96,212.77,112.69,374.99,509.57,112.67,144.00,622.06,682.36,613.48,...,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30
2022-12-02 14:00:00,212.77,112.69,374.99,509.57,112.67,144.00,622.06,682.36,613.48,233.12,...,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30,527.70
2022-12-02 15:00:00,112.69,374.99,509.57,112.67,144.00,622.06,682.36,613.48,233.12,622.65,...,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30,527.70,645.20


In [25]:
selected_cols = ['ail', 'gas_reserve_margin', 'wind_reserve_margin', 'other_reserve_margin', 'load_on_gas_reserve', 'gas_price', 'gas_supply_mix', 'demand_supply_ratio', 'avail_gen_ratio', 'fossil_fuel_ratio', 'gas_tng_ratio']
X_features = price_old_df[selected_cols]
data_df = pd.merge(price_data_df, X_features, left_index=True, right_index=True)
data_df.head(2)

,price_lag_35,price_lag_34,price_lag_33,price_lag_32,price_lag_31,price_lag_30,price_lag_29,price_lag_28,price_lag_27,price_lag_26,...,gas_reserve_margin,wind_reserve_margin,other_reserve_margin,load_on_gas_reserve,gas_price,gas_supply_mix,demand_supply_ratio,avail_gen_ratio,fossil_fuel_ratio,gas_tng_ratio
2022-12-02 11:00:00,87.67,108.46,104.96,212.77,112.69,374.99,509.57,112.67,144.00,622.06,...,-0.014491,0.903325,-0.630735,-0.014101,5.65,0.736310,0.820136,1.235130,0.809045,0.736310
2022-12-02 12:00:00,108.46,104.96,212.77,112.69,374.99,509.57,112.67,144.00,622.06,682.36,...,-0.018357,0.919257,-0.669212,-0.017903,5.65,0.738659,0.815413,1.234867,0.811501,0.738659


In [26]:
x_feature_names = [f"price_lag_{i}" for i in range(1, window_length+1)]
x_feature_names += selected_cols
x_feature_names

['price_lag_1',
 'price_lag_2',
 'price_lag_3',
 'price_lag_4',
 'price_lag_5',
 'price_lag_6',
 'price_lag_7',
 'price_lag_8',
 'price_lag_9',
 'price_lag_10',
 'price_lag_11',
 'price_lag_12',
 'price_lag_13',
 'price_lag_14',
 'price_lag_15',
 'price_lag_16',
 'price_lag_17',
 'price_lag_18',
 'price_lag_19',
 'price_lag_20',
 'price_lag_21',
 'price_lag_22',
 'price_lag_23',
 'price_lag_24',
 'price_lag_25',
 'price_lag_26',
 'price_lag_27',
 'price_lag_28',
 'price_lag_29',
 'price_lag_30',
 'price_lag_31',
 'price_lag_32',
 'price_lag_33',
 'price_lag_34',
 'price_lag_35',
 'gas_reserve_margin',
 'wind_reserve_margin',
 'other_reserve_margin',
 'load_on_gas_reserve',
 'gas_price',
 'gas_supply_mix',
 'demand_supply_ratio',
 'avail_gen_ratio',
 'fossil_fuel_ratio',
 'gas_tng_ratio']

In [27]:
X = data_df[x_feature_names]
X.head(2)

,price_lag_1,price_lag_2,price_lag_3,price_lag_4,price_lag_5,price_lag_6,price_lag_7,price_lag_8,price_lag_9,price_lag_10,...,gas_reserve_margin,wind_reserve_margin,other_reserve_margin,load_on_gas_reserve,gas_price,gas_supply_mix,demand_supply_ratio,avail_gen_ratio,fossil_fuel_ratio,gas_tng_ratio
2022-12-02 11:00:00,178.05,340.44,762.16,574.26,760.57,120.79,596.05,187.26,106.89,150.99,...,-0.014491,0.903325,-0.630735,-0.014101,5.65,0.736310,0.820136,1.235130,0.809045,0.736310
2022-12-02 12:00:00,181.82,178.05,340.44,762.16,574.26,760.57,120.79,596.05,187.26,106.89,...,-0.018357,0.919257,-0.669212,-0.017903,5.65,0.738659,0.815413,1.234867,0.811501,0.738659


In [28]:
y = data_df['price_target']
y.head(2)

2022-12-02 11:00:00    181.82
2022-12-02 12:00:00    337.13
Name: price_target, dtype: float64

In [29]:
X.shape[0] == y.shape[0]   # same number of rows. good!

True

In [30]:
test_size = 48
forcast_len = 1
total_forecast_len = 12

y_train, y_test_full, X_train, X_test = temporal_train_test_split(y, X, test_size=test_size+total_forecast_len)

y_test = y_test_full[:-total_forecast_len] # We need X for the last forcast_len hours to make predictions.

fh = ForecastingHorizon(np.arange(1, forcast_len+1))

y_train = y_train.asfreq('H')
X_train = X_train.asfreq('H')
X_test = X_test.asfreq('H')
y_test = y_test.asfreq('H')

In [31]:
y_train.shape[0] == X_train.shape[0]

True

In [32]:
y_test.tail(1)  # We will update the model with this value of y_test and X_test and predict for the next 12 hours.
# Y test full should have actual values for the these 12 hours.
# y_test_full[0] will be one step prediction, y_test_full[1] will be 2 step prediction and so on.

2023-03-31 10:00:00    49.26
Freq: H, Name: price_target, dtype: float64

In [33]:
y_test_full.tail(1)

2023-03-31 22:00:00    46.38
Name: price_target, dtype: float64

In [34]:
models = {}
X_sub_train = {}

# Create a for loop
for j in range(1, len_models + 1):
    # Generate a unique model name
    model_name = f"model_{j}"
    X_train_name = f"X_train_{j}"

    lag_cols = [f"price_lag_{i}" for i in range(j, j + num_lags)]

    X_sub_train[X_train_name] = X_train[lag_cols + selected_cols].copy()
    X_sub_train[X_train_name] = X_sub_train[X_train_name].asfreq('H')
    
    # Create the pipeline for each model
    models[model_name] = ForecastingPipeline(
        steps=[
            ("price_column_transformer", ColumnwiseTransformer(LogTransformer(), columns=lag_cols)),
            ("ail_column_transformer", ColumnwiseTransformer(Deseasonalizer(sp=24, model="additive"), columns=['ail'])),
            (
                "forecaster",
                TransformedTargetForecaster(
                    steps=[
                        ("targetlogtransformer", LogTransformer()),
                        ("targetforecaster", YfromX(RandomForestRegressor(n_estimators=200, n_jobs=-1))),
                    ]
                ),
            ),
        ]
    )
    
    models[model_name].fit(y=y_train, X=X_sub_train[X_train_name], fh=fh)

In [35]:
predictions_df = pd.DataFrame(columns=[f"cutoff_hour_{models['model_1'].cutoff.hour[0]}"])
predictions_df

,cutoff_hour_10


In [36]:
X_sub_test = {}

for j in range(1, len_models + 1):
    model_name = f"model_{j}"
    lag_cols = [f"price_lag_{i}" for i in range(j, j + num_lags)]
    X_test_name = f"X_test_{j}"
    
    X_sub_test[X_test_name] = X_test[lag_cols + selected_cols].copy()
    X_sub_test[X_test_name] = X_sub_test[X_test_name].asfreq('H')

    cutoff_time = models[model_name].cutoff
    prediction_for = cutoff_time + pd.DateOffset(hours=j)

    y_pred = models[model_name].predict(fh = 1, X=X_sub_test[X_test_name][0:forcast_len])
    
    row = pd.DataFrame({f"cutoff_hour_{models['model_1'].cutoff.hour[0]}": y_pred[0]}, index=pd.Index(prediction_for))
    
    predictions_df = predictions_df.append(row)

In [37]:
predictions_df.index.name = 'Time'
predictions_df

,cutoff_hour_10
Time,
2023-03-29 11:00:00,62.621509
2023-03-29 12:00:00,67.248107
2023-03-29 13:00:00,64.143240
2023-03-29 14:00:00,64.761434
2023-03-29 15:00:00,64.765050
2023-03-29 16:00:00,64.051896
2023-03-29 17:00:00,63.805643
2023-03-29 18:00:00,64.437369
2023-03-29 19:00:00,62.925850


In [38]:
rolling_prediction_df = pd.DataFrame(index=y_test_full.index)
rolling_prediction_df = pd.concat([rolling_prediction_df, predictions_df], axis=1)
rolling_prediction_df

,cutoff_hour_10
2023-03-29 11:00:00,62.621509
2023-03-29 12:00:00,67.248107
2023-03-29 13:00:00,64.143240
2023-03-29 14:00:00,64.761434
2023-03-29 15:00:00,64.765050
2023-03-29 16:00:00,64.051896
2023-03-29 17:00:00,63.805643
2023-03-29 18:00:00,64.437369
2023-03-29 19:00:00,62.925850
2023-03-29 20:00:00,64.334790


In [39]:
for j in range(1, len_models + 1):
    # print cut_off time for each model
    print(models[f"model_{j}"].cutoff)

DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')


In [40]:
# emulating the rolling prediction for the next hours

for i in range(0, len(y_test)): # Loop through the test set emulating the passing by of one hour. 
        
        predictions_df = pd.DataFrame()
        predictions_df
        
        for j in range(1, len_models + 1): # As each hour passes, update 12 models.
                model_name = f"model_{j}"
                X_test_name = f"X_test_{j}"
                
                new_observation_y, new_observation_X  = y_test[i:i+1], X_sub_test[X_test_name][i:i+1]
        
                new_observation_y = new_observation_y.asfreq('H')
                new_observation_X = new_observation_X.asfreq('H')

                print(f'Updating model_{j} with actual values at {new_observation_y.index[0]}')

                print(f'Cut off before update: {models[model_name].cutoff}')

                models[model_name].update(y=new_observation_y, X=new_observation_X, update_params=False)

                print(f'Cut off after update: {models[model_name].cutoff}')

                cutoff_time = models[model_name].cutoff
                prediction_for = cutoff_time + pd.DateOffset(hours=j)

                print(f'Predicting for {prediction_for} using model_{j} with X at {X_sub_test[X_test_name][(i+1):(i+forcast_len+1)].index[0]}')
                
                y_pred = models[model_name].predict(fh, X=X_sub_test[X_test_name][(i+1):(i+forcast_len+1)])

                row = pd.DataFrame({f"cutoff_hour_{models['model_1'].cutoff.hour[0]}": y_pred[0]}, index=pd.Index(prediction_for))
                
                predictions_df = predictions_df.append(row)
        
        rolling_prediction_df = pd.concat([rolling_prediction_df, predictions_df], axis=1)
        print(f'Update and prediction done for {new_observation_y.index[0]}')
        print(f'----------------------------------------------------------------------------------')

Updating model_1 with actual values at 2023-03-29 11:00:00
Cut off before update: DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
Cut off after update: DatetimeIndex(['2023-03-29 11:00:00'], dtype='datetime64[ns]', freq='H')
Predicting for DatetimeIndex(['2023-03-29 12:00:00'], dtype='datetime64[ns]', freq=None) using model_1 with X at 2023-03-29 12:00:00
Updating model_2 with actual values at 2023-03-29 11:00:00
Cut off before update: DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
Cut off after update: DatetimeIndex(['2023-03-29 11:00:00'], dtype='datetime64[ns]', freq='H')
Predicting for DatetimeIndex(['2023-03-29 13:00:00'], dtype='datetime64[ns]', freq=None) using model_2 with X at 2023-03-29 12:00:00
Updating model_3 with actual values at 2023-03-29 11:00:00
Cut off before update: DatetimeIndex(['2023-03-29 10:00:00'], dtype='datetime64[ns]', freq='H')
Cut off after update: DatetimeIndex(['2023-03-29 11:00:00'], dtype='datetime64[

In [41]:
# rolling_prediction_df

In [42]:
# All cut off times should be the updated and same. We can see that all models have been updated to the last value of y_test.
for j in range(1, len_models + 1):
    # print cut_off time for each model
    print(models[f"model_{j}"].cutoff)

DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')
DatetimeIndex(['2023-03-31 10:00:00'], dtype='datetime64[ns]', freq='H')


In [43]:
y_test.tail()

2023-03-31 06:00:00    48.86
2023-03-31 07:00:00    51.07
2023-03-31 08:00:00    49.35
2023-03-31 09:00:00    47.94
2023-03-31 10:00:00    49.26
Freq: H, Name: price_target, dtype: float64

In [44]:
y_test_full.tail()

2023-03-31 18:00:00    50.86
2023-03-31 19:00:00    49.35
2023-03-31 20:00:00    51.28
2023-03-31 21:00:00    48.39
2023-03-31 22:00:00    46.38
Name: price_target, dtype: float64

In [45]:
y_test.index[0]

Timestamp('2023-03-29 11:00:00', freq='H')

In [46]:
rolling_prediction_df.head(1) # Should start from Y_test.index[0]

,cutoff_hour_10,cutoff_hour_11,cutoff_hour_12,cutoff_hour_13,cutoff_hour_14,cutoff_hour_15,cutoff_hour_16,cutoff_hour_17,cutoff_hour_18,cutoff_hour_19,...,cutoff_hour_1,cutoff_hour_2,cutoff_hour_3,cutoff_hour_4,cutoff_hour_5,cutoff_hour_6,cutoff_hour_7,cutoff_hour_8,cutoff_hour_9,cutoff_hour_10
2023-03-29 11:00:00,62.621509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
y_test_full.index[-1]

Timestamp('2023-03-31 22:00:00')

In [48]:
rolling_prediction_df.tail(1) # Should end with y_test_full.index[-1]

,cutoff_hour_10,cutoff_hour_11,cutoff_hour_12,cutoff_hour_13,cutoff_hour_14,cutoff_hour_15,cutoff_hour_16,cutoff_hour_17,cutoff_hour_18,cutoff_hour_19,...,cutoff_hour_1,cutoff_hour_2,cutoff_hour_3,cutoff_hour_4,cutoff_hour_5,cutoff_hour_6,cutoff_hour_7,cutoff_hour_8,cutoff_hour_9,cutoff_hour_10
2023-03-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.579414


In [49]:
import pandas as pd
import numpy as np

def generate_step_predictions(rolling_prediction_df, y_test_full, num_steps):
    step_predictions = []
    
    for step in range(0, num_steps):
        diag_values = np.diag(rolling_prediction_df.values, -step)
        
        index_range = y_test_full.index[step:step + len(diag_values)]
        column_name = f'{step+1}_step_prediction'
        
        prediction_df = pd.DataFrame(diag_values, index=index_range, columns=[column_name])
        
        if y_test_full[step:step + len(prediction_df)].index.equals(prediction_df.index):
            step_predictions.append(prediction_df)
        else:
            print(f"Error: Index mismatch for {step}-step prediction.")
    
    return step_predictions

In [50]:
predictions = generate_step_predictions(rolling_prediction_df, y_test_full, total_forecast_len)

In [51]:
# predictions[0] # 1 initial prediction + 48 steps of update and prediction from the test set.

In [52]:
# predictions[1] # 1 initial prediction + 48 steps of update and prediction from the test set.

In [53]:
# predictions[2] # 1 initial prediction + 48 steps of update and prediction from the test set.

In [54]:
step_sizes = np.arange(1, total_forecast_len+1)
for step, prediction_series in zip(step_sizes, predictions):
    if y_test_full[step-1:step+test_size].index.equals(prediction_series.index):
        rmse = mean_squared_error(y_test_full[step-1:step+test_size], prediction_series, squared=False)
        print(f"{step} Step RMSE for model: {rmse}")

1 Step RMSE for model: 20.96594069308167
2 Step RMSE for model: 26.7508160176403
3 Step RMSE for model: 26.02371499580435
4 Step RMSE for model: 27.417062643250333
5 Step RMSE for model: 29.65075532188151
6 Step RMSE for model: 29.66849820941314
7 Step RMSE for model: 30.86207162967737
8 Step RMSE for model: 32.51915115715713
9 Step RMSE for model: 31.793994228862026
10 Step RMSE for model: 31.23382193804961
11 Step RMSE for model: 26.09964508259169
12 Step RMSE for model: 19.063356890061854
